In [1]:
import numpy as np
import pandas as pd
import random
from dotenv import dotenv_values, find_dotenv
import os


# set path parameters
config = dotenv_values(find_dotenv())
path_rawdata = os.path.abspath(config["RAWDATA"]) + '\\'
path_cleandata = os.path.abspath(config["CLEANDATA"]) + '\\'
path_figures = os.path.abspath(config["FIGURES"]) + '\\'


In [13]:
import numpy as np
import pandas as pd
import random
from dotenv import dotenv_values, find_dotenv
import os

# set path parameters
config = dotenv_values(find_dotenv())
path_rawdata = os.path.abspath(config["RAWDATA"]) + '\\'
path_cleandata = os.path.abspath(config["CLEANDATA"]) + '\\'
path_figures = os.path.abspath(config["FIGURES"]) + '\\'

# import I-O shares
intermediate_costshares = pd.read_pickle(path_cleandata + 'inversions//intermediate_costshares.pkl')
intermediate_salesshares = pd.read_pickle(path_cleandata + 'inversions//intermediate_salesshares.pkl')

# filter for products
products_to_include = list(intermediate_costshares.index)
products_to_include.sort()
intermediate_salesshares = intermediate_salesshares[intermediate_salesshares.index.isin(products_to_include)]['intermediate_salesshare']

# get list of dates to use (just comes from whatevers available in the BEA data)
prices = pd.read_pickle(path_cleandata + 'inversions//prices.pkl')
quantities = pd.read_pickle(path_cleandata + 'inversions//quantities.pkl')
expenditures = pd.read_pickle(path_cleandata + 'inversions//expenditures.pkl')
dates = list(set(prices['date'].unique()) & set(quantities['date'].unique()) & set(expenditures['date'].unique()))
dates.sort()

# monte carlo parameters

random.seed(420)

# initial shock parameters
sd = 0.01
supplyshock_params = np.random.uniform(-0.02, 0.02, size=len(products_to_include))
demandshock_params = np.random.uniform(-0.02, 0.02, size=len(products_to_include))

# supply curve
alpha = 1.2

# price generation convergence
convergence_threshold = 1e-7
max_change = float('inf')

# initial values

# supply and demand shocks
shocks_generated = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'supply_shock': pd.Series(dtype='float'),
                   'demand_shock': pd.Series(dtype='float')})
for product in products_to_include:
    initial_shock = pd.DataFrame([[dates[0], product, 0.0, 0.0]], columns=shocks_generated.columns)
    shocks_generated = pd.concat([shocks_generated, initial_shock])


# random walk
store_randomwalk_supply = [0]
store_randomwalk_demand = [0]

# fill in fake supply and demand
for date in dates[1:]:
    # create new random
    randomwalk_supply = np.random.normal(loc=0, scale=sd, size=len(products_to_include))
    store_randomwalk_supply.append(randomwalk_supply)
    randomwalk_demand = np.random.normal(loc=0, scale=sd, size=len(products_to_include))
    store_randomwalk_demand.append(randomwalk_demand)

    # initial shock
    shocks_date = shocks_generated.loc[shocks_generated['date'] == dates[dates.index(date)-1]][['product', 'supply_shock', 'demand_shock']]

    # current shock = previous shock + (parameter * previous random + current random)
    shocks_date['supply_shock'] = shocks_date['supply_shock'] + (supplyshock_params * store_randomwalk_supply[dates.index(date)-1] + randomwalk_supply)
    shocks_date['demand_shock'] = shocks_date['demand_shock'] + (demandshock_params * store_randomwalk_demand[dates.index(date)-1] + randomwalk_demand)
    
    shocks_date['date'] = date

    shocks_generated = pd.concat([shocks_generated, shocks_date])


# # so i had the wrong order of operations here before (i was taking e^shock every time which is wrong)
shocks_generated['supply_shock'] = np.exp(shocks_generated['supply_shock'])
shocks_generated['demand_shock'] = np.exp(shocks_generated['demand_shock'])

shocks_generated


,date,product,supply_shock,demand_shock
0,1959-01-31,Accessories and parts,1.000000,1.000000
0,1959-01-31,Air transportation,1.000000,1.000000
0,1959-01-31,Alcohol in purchased meals,1.000000,1.000000
0,1959-01-31,"Amusement parks, campgrounds, and related recr...",1.000000,1.000000
0,1959-01-31,"Audio discs, tapes, vinyl, and permanent digit...",1.000000,1.000000
...,...,...,...,...
0,2023-07-31,Water supply and sewage maintenance,0.920308,0.805961
0,2023-07-31,Water transportation,1.263384,0.985494
0,2023-07-31,Window coverings,0.789477,0.834009
0,2023-07-31,Wine,1.030928,0.748024


In [10]:
numerical_columns = shocks_generated.select_dtypes(include=['number']).columns

# Filter rows where any numerical column has a value less than 0
filtered_df = shocks_generated[shocks_generated[numerical_columns].lt(0).any(axis=1)]

print(filtered_df)

         date                                            product  \
0  1959-04-30                              Accessories and parts   
0  1959-04-30                                 Air transportation   
0  1959-04-30                         Alcohol in purchased meals   
0  1959-04-30  Amusement parks, campgrounds, and related recr...   
0  1959-04-30  Audio discs, tapes, vinyl, and permanent digit...   
..        ...                                                ...   
0  2023-07-31                Water supply and sewage maintenance   
0  2023-07-31                               Water transportation   
0  2023-07-31                                   Window coverings   
0  2023-07-31                                               Wine   
0  2023-07-31                        Women's and girls' clothing   

    supply_shock  demand_shock  
0      -0.019067     -0.001552  
0      -0.001468     -0.001537  
0      -0.009981      0.018700  
0      -0.005562     -0.014951  
0      -0.003919  

In [30]:
shocks_generated["classification"] = shocks_generated.apply(
    lambda row: "positive demand" if (row["demand_shock"] > 0 and row["supply_shock"] < 0 and abs(row["demand_shock"]) > abs(row["supply_shock"])) 
                else ("negative demand" if (row["demand_shock"] < 0 and row["supply_shock"] > 0 and abs(row["demand_shock"]) > abs(row["supply_shock"]))
                else np.nan),
    axis=1
)

nans = shocks_generated[shocks_generated['classification'].isna()]
nans.to_excel("nans.xlsx")